In [1]:
import numpy as np
import pandas as pd


In [25]:
A = np.load("data_A.npy")
y = np.load("labels.npy")

In [30]:
np.save("input_x",A[0:2])

In [29]:
np.save("X",A[2:])

In [59]:
C = C.reshape((C.shape[0],3))[:,[0,2]]

In [18]:
valid_mask = np.isfinite(A_).all(axis=-1)

# Filter out rows where C has inf or NaN values
filtered_A = A[valid_mask]
filtered_y = y[valid_mask]

In [61]:
np.save("data_A",filtered_A)
np.save("data_B",filtered_B)
np.save("data_C",filtered_C)
np.save("labels",filtered_y)

In [42]:
len(filtered_A)

161823

In [40]:
len(C)

161901

In [39]:
len(C[np.isfinite(C).all(axis=1)])

161823

In [36]:
np.mean(C,axis=0)

array([       nan, 0.49577771])

In [31]:
C.mean()

nan

In [25]:
df_C = pd.DataFrame(C)

In [26]:
df_C.describe()

,0,1
count,161823.000000,161901.000000
mean,14.283578,0.495778
std,0.650797,0.288871
min,12.170000,0.000000
25%,13.800000,0.245283
50%,14.330000,0.495327
75%,14.750000,0.746835
max,16.300000,0.992806


In [29]:
df_C[df_C.isnull()]

,0,1
0,NaN,NaN
1,NaN,NaN
2,NaN,NaN
3,NaN,NaN
4,NaN,NaN
...,...,...
161896,NaN,NaN
161897,NaN,NaN
161898,NaN,NaN
161899,NaN,NaN


In [22]:
len(y[y==1])

79021

In [17]:
A.shape

(161901, 14, 12)

In [18]:
B.shape

(161901, 6, 2)

In [5]:
A.shape

(161901, 14, 12)

In [16]:
C.shape

(161901, 2)

In [39]:
A2 = np.load("data_A_2023.npy")
A2 = normalize_data(A2)
B2 = np.load("data_B_2023.npy")
B2 = normalize_data(B2)
y2 = np.load("labels_2023.npy")

In [40]:
A_new = np.concatenate((A,A2))
B_new = np.concatenate((B,B2))
y_new = np.concatenate((y,y2))

In [42]:
A_new.shape

(207754, 14, 12)

In [44]:
B_new.shape

(207754, 6, 2)

In [41]:
np.save("data_A_combined",A_new)
np.save("data_B_combined",B_new)
np.save("labels_combined",y_new)

In [15]:
from .. import utils

ImportError: attempted relative import with no known parent package

In [5]:
B.shape

(75322, 6, 2)

In [7]:
y.shape

(75322, 1)

In [6]:
len(y[y==0])

65313

In [204]:
print(df1.head())
print(df2.head())

   00:00  03:00  06:00  09:00  12:00  15:00  18:00  21:00        Date
0  27 °c  25 °c  25 °c  30 °c  31 °c  28 °c  27 °c  26 °c  2022-03-01
1   9 °c   8 °c   8 °c  10 °c  13 °c  17 °c  15 °c  10 °c  2022-03-02
2   8 °c   8 °c   8 °c  12 °c  16 °c  18 °c  15 °c  13 °c  2022-03-03
3  13 °c  12 °c  12 °c  13 °c  15 °c  16 °c  15 °c  11 °c  2022-03-04
4  11 °c  11 °c  10 °c  13 °c  16 °c  17 °c  12 °c   7 °c  2022-03-05
          日期   时间    top   bot     y pressure
1 2023-03-22   8点  114.9  50.8  7.63     3.75
2 2023-03-22   9点    116    51     —     3.75
3 2023-03-22  10点  116.5  51.1     8     3.76
4 2023-03-22  11点    117  51.4     —     3.75
5 2023-03-22  12点  117.1  51.5     8     3.76


In [2]:
import pandas as pd
import numpy as np

df = pd.read_excel('industry.xlsx')
df['日期'] = df['日期'].fillna(method='ffill')
df = df.drop(df.columns[[1, 8, 9]], axis=1)
df = df.iloc[1:]

new_column_names = ["日期","时间","top","bot","y","pressure","flow_in"]
# Create a dictionary to map the current column names to the new column names
column_mapping = {current_name: new_name for current_name, new_name in zip(df.columns, new_column_names)}

# Rename the columns using the dictionary
df.rename(columns=column_mapping, inplace=True)

df1 = pd.read_csv("../weather_data.csv").dropna()
df2 = df.dropna()

# clean df1
df1 = df1.melt(id_vars='Date', var_name='Time', value_name='Temp')
df1['Temp'] = df1['Temp'].str.replace(' °c','').astype(int)
df1['Time'] = pd.to_datetime(df1['Time'], format='%H:%M').dt.strftime('%#H:%M')
df1['Date'] = pd.to_datetime(df1['Date'])  # Convert to datetime

# convert 'Time' to timedelta
df1['Time'] = pd.to_timedelta(df1['Time'] + ':00')

# sort by 'Date' and 'Time'
df1 = df1.sort_values(['Date', 'Time'])

# Convert 'Time' back to string format without leading zero
df1['Time'] = df1['Time'].dt.components.hours.astype(str) + ':' + df1['Time'].dt.components.minutes.astype(str).str.zfill(2)

# assuming df is your DataFrame

# Combine 'Date' and 'Time' into a single datetime column
df1['DateTime'] = pd.to_datetime(df1['Date'].astype(str) + ' ' + df1['Time'])  # Convert 'Date' to string before appending
df1 = df1.set_index('DateTime')

# Create a new DateTimeIndex that includes all hours
new_index = pd.date_range(start=df1.index.min(), end=df1.index.max(), freq='H')

# Reindex the DataFrame and interpolate missing values
df1 = df1.reindex(new_index)
df1['Temp'] = df1['Temp'].interpolate(method='linear')

# Split the datetime index back into separate date and time columns
df1['Date'] = df1.index.date
df1['Time'] = df1.index.time

df1['Time'] = df1['Time'].astype(str).str[:-3]
df1['Time'] = pd.to_datetime(df1['Time'],format='%H:%M').dt.strftime('%#H:%M')
df1['Date'] = pd.to_datetime(df1['Date'])  # Convert to datetime

# Convert 'Temp' back to integer
df1['Temp'] = df1['Temp'].round().astype(int)

# clean df2
df2['Time'] = df2['时间'].str.replace('点',':00')
df2 = df2.rename(columns={'日期': 'Date'})
df2['Time'] = pd.to_datetime(df2['Time'],format='%H:%M').dt.strftime('%#H:%M')
df2['Date'] = pd.to_datetime(df2['Date'])  # Convert to datetime
# merge dataframes
df = pd.merge(df2, df1, on=['Date', 'Time'], how='left')
df.dropna(inplace=True)


C:\Users\x00651573\AppData\Local\Temp\ipykernel_9620\171399081.py:59: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2['Time'] = df2['时间'].str.replace('点',':00')


In [ ]:
df = df[~(df == '—').any(axis=1)]

In [10]:
df

,Date,时间,top,bot,y,pressure,flow_in,Time,Temp
0,2023-03-22,8点,114.9,50.8,7.63,3.75,35,8:00,17.0
2,2023-03-22,10点,116.5,51.1,8,3.76,35,10:00,18.0
4,2023-03-22,12点,117.1,51.5,8,3.76,35,12:00,17.0
6,2023-03-22,14点,117.1,51.3,8.11,3.9,36,14:00,16.0
8,2023-03-22,16点,117.6,50.6,8.72,3.96,36,16:00,15.0
...,...,...,...,...,...,...,...,...,...
2325,2023-06-28,6点,118.9,53.9,7.67,3.36,35,6:00,24.0
2327,2023-06-28,8点,118.9,54,7.83,3.39,35,8:00,28.0
2329,2023-06-28,10点,118.3,54.6,7.58,3.48,35,10:00,32.0
2331,2023-06-28,12点,118.4,54.9,7.64,3.4,35,12:00,37.0


In [33]:
df_w = pd.read_excel('water.xlsx',names=["date1","time1","date2","time2","water"])

# assuming df is your DataFrame

df_w['time1'] = df_w['time1'].astype(str)
df_w['time2'] = df_w['time2'].astype(str)

df_w['start'] = pd.to_datetime(df_w['date1'].astype(str) + ' ' + df_w['time1'])
df_w['end'] = pd.to_datetime(df_w['date2'].astype(str) + ' ' + df_w['time2'])

# create an empty DataFrame to store the results
df_w_new = pd.DataFrame()

for i in range(len(df_w)):
    # create a datetime range for each row
    r = pd.date_range(start=df_w.loc[i, 'start'], end=df_w.loc[i, 'end'], freq='H')

    # create a new DataFrame for this range
    df_temp = pd.DataFrame({
        'Date': r.date,
        'Time': r.time,
        'Water': df_w.loc[i, 'water']
    })

    # append this DataFrame to the results DataFrame
    df_w_new = pd.concat([df_w_new, df_temp], ignore_index=True)

# Convert 'Time' back to string format without leading zero
df_w_new['Time'] = df_w_new['Time'].apply(lambda x: x.strftime('%#H:%M'))

df_w_new['Time']  = pd.to_datetime (df_w_new['Time'],format='%H:%M').dt.strftime('%#H:%M')
df_w_new['Date']  = pd.to_datetime (df_w_new['Date'])

In [36]:
df = pd.merge(df, df_w_new, on=['Date', 'Time'], how='left')

In [41]:
df.dropna(inplace=True)

In [42]:
X = df[["top","bot","pressure","Temp","flow_in","Water"]].values.astype('float32')
y = df[["y"]].values.astype('float32')
N,K = X.shape

In [44]:
np.save('X.npy', X.reshape(N,1,K))
np.save('y.npy', y)


In [19]:
X_ = np.load("X.npy",allow_pickle= True)
y_ = np.load("y.npy",allow_pickle= True)

In [46]:
len(X)

954